### Imports

In [1]:
import numpy as np
import pandas as pd

### Read Data Sets

In [73]:
types = {'SP_NAIH': str, 'SP_ATOPROF': str, 'SP_CIDPRI': str, 'SP_QTD_ATO': np.int32}

#### CIDS

In [56]:
o_cids = pd.read_csv('../data/o_cids.csv', sep=';')
o_cids.head()

,influence_type,code,description
0,Morte Materna Direta,O000,Gravidez abdominal
1,Morte Materna Direta,O001,Gravidez tubária
2,Morte Materna Direta,O002,Gravidez ovariana
3,Morte Materna Direta,O008,Outras formas de gravidez ectópica
4,Morte Materna Direta,O009,"Gravidez ectópica, não especificada"


#### Procedures

In [71]:
procedures = pd.read_csv('../data/procedimentos_obstetricia.csv', sep=';')
procedures.head()

,CODE,DESCRIPTION
0,205020143,Ultrassonografia obstétrica
1,205020186,Ultrassonografia transvaginal
2,205010059,Ultrassonografia doppler de fluxo obstétrico
3,205020151,Ultrassonografia obstétrica com doppler colori...
4,201010011,Amniocentese


#### March

In [74]:
df_sp_mar = pd.read_csv('../../SPSP/SPSP1803.csv', sep=',', usecols=types.keys(), dtype=types)
df_sp_mar["SP_ATOPROF"] = df_sp_mar["SP_ATOPROF"].str[1:]
df_sp_mar.head()

,SP_NAIH,SP_ATOPROF,SP_QTD_ATO,SP_CIDPRI
0,3518100401101,303010061,1,A09
1,3518100401101,301010170,8,A09
2,3518100401101,301010170,1,A09
3,3518100401101,301010170,1,A09
4,3518100401101,802010040,1,A09


In [75]:
df_sp_mar.shape

(3081817, 4)

##### Filter CID 

In [76]:
df_sp_mar_cids = df_sp_mar.merge(o_cids, left_on='SP_CIDPRI', right_on='code', how='left').drop('code', axis=1)

df_sp_mar_cids = df_sp_mar_cids[df_sp_mar_cids['influence_type'].notna()]

In [77]:
df_sp_mar_cids.head()

,SP_NAIH,SP_ATOPROF,SP_QTD_ATO,SP_CIDPRI,influence_type,description
196,3518105143696,303100044,1,O998,Morte Materna Indireta,Outras doenças e afecções especificadas compli...
257,3518105143564,303100044,1,O16,Morte Materna Direta,Hipertensão materna não especificada
311,3518105143564,202010473,1,O16,Morte Materna Direta,Hipertensão materna não especificada
314,3518105144048,303100044,1,O998,Morte Materna Indireta,Outras doenças e afecções especificadas compli...
315,3518105148492,303100044,1,O479,Morte Materna Direta,"Falso trabalho de parto, não especificado"


##### Filter Procedures

In [78]:
df_sp_mar_proc = df_sp_mar.merge(procedures, left_on='SP_ATOPROF', right_on='CODE', how='left').drop('CODE', axis=1)
df_sp_mar_proc = df_sp_mar_proc[df_sp_mar_proc['DESCRIPTION'].notna()]

In [79]:
df_sp_mar_proc.head()

,SP_NAIH,SP_ATOPROF,SP_QTD_ATO,SP_CIDPRI,DESCRIPTION
196,3518105143696,303100044,1,O998,Tratamento de intercorrências clínicas na grav...
257,3518105143564,303100044,1,O16,Tratamento de intercorrências clínicas na grav...
314,3518105144048,303100044,1,O998,Tratamento de intercorrências clínicas na grav...
315,3518105148492,303100044,1,O479,Tratamento de intercorrências clínicas na grav...
316,3518105148350,303100044,1,O998,Tratamento de intercorrências clínicas na grav...


#### April

In [80]:
df_sp_apr = pd.read_csv('../../SPSP/SPSP1804.csv', sep=',', usecols=types.keys(), dtype=types)
df_sp_apr["SP_ATOPROF"] = df_sp_apr["SP_ATOPROF"].str[1:]
df_sp_apr.head()

,SP_NAIH,SP_ATOPROF,SP_QTD_ATO,SP_CIDPRI
0,3518102098456,303010061,1,A09
1,3518102098456,802010083,5,A09
2,3518102098456,206030010,1,A09
3,3518102098456,206030037,1,A09
4,3518102098456,603070019,1,A09


##### Filter CIDS

In [81]:
df_sp_apr_cids = df_sp_apr.merge(o_cids, left_on='SP_CIDPRI', right_on='code', how='left').drop('code', axis=1)
df_sp_apr_cids = df_sp_apr_cids[df_sp_apr_cids['influence_type'].notna()]

In [82]:
df_sp_apr_cids.head()

,SP_NAIH,SP_ATOPROF,SP_QTD_ATO,SP_CIDPRI,influence_type,description
3922,3518114318114,202010228,4,O140,Morte Materna Direta,Pré-eclâmpsia moderada
3923,3518114318114,202010317,8,O140,Morte Materna Direta,Pré-eclâmpsia moderada
3924,3518114318114,202010368,7,O140,Morte Materna Direta,Pré-eclâmpsia moderada
3925,3518114318114,202010422,3,O140,Morte Materna Direta,Pré-eclâmpsia moderada
3926,3518114318114,202010465,3,O140,Morte Materna Direta,Pré-eclâmpsia moderada


In [34]:
df_sp_apr_cids.shape

(565244, 6)

##### Filter Procedures

In [83]:
df_sp_apr_proc = df_sp_apr.merge(procedures, left_on='SP_ATOPROF', right_on='CODE', how='left').drop('CODE', axis=1)
df_sp_apr_proc = df_sp_apr_proc[df_sp_apr_proc['DESCRIPTION'].notna()]

In [85]:
df_sp_apr_proc.head()

,SP_NAIH,SP_ATOPROF,SP_QTD_ATO,SP_CIDPRI,DESCRIPTION
3943,3518114318169,303100044,1,O244,Tratamento de intercorrências clínicas na grav...
3946,3518114318169,205020143,1,O244,Ultrassonografia obstétrica
3949,3518114318170,411010026,1,O418,Parto cesariano em gestação de alto risco
3950,3518114318170,411010026,1,O418,Parto cesariano em gestação de alto risco
3951,3518114318170,417010036,1,O418,Anestesia Obstétrica p/ Cesariana em Gestaça...


#### RD

In [86]:
aih_types = {'N_AIH': str, 'SEXO': np.int32, 'IDADE': np.int32, 'INSTRU': np.int32}

In [87]:
df_rd = pd.read_csv('../../AIH/sih_sus_m_10_49.csv', sep='|', usecols=aih_types.keys(), dtype=aih_types)
df_rd.head()

,N_AIH,SEXO,IDADE,INSTRU
0,3518101448609,3,10,0
1,3518101279825,3,47,0
2,3518101457475,3,25,0
3,3518101457662,3,49,0
4,3518101525246,3,14,0


### Concatenate March and April

In [88]:
# como as tabelas resultantes do filtro de procedimentos não retornaram valores, vamos concatenar apenas as de cid
df_concat_sp_mar_apr = pd.concat([df_sp_mar_cids, df_sp_mar_proc, df_sp_apr_cids, df_sp_apr_proc])
df_concat_sp_mar_apr.shape

(1471357, 7)

In [90]:
df_concat_sp_mar_apr.drop_duplicates(inplace=True)

In [35]:
df_sp_mar_cids.shape[0] + df_sp_apr_cids.shape[0]

1133944

In [91]:
del df_sp_mar
del df_sp_apr

### Left Merge between SP and RD datasets

In [92]:
# merged_dfs = sp_df.merge(o_cids, left_on='SP_CIDPRI', right_on='code', how='left').drop('code', axis=1)
merged_sp_rd = df_concat_sp_mar_apr.merge(df_rd, left_on='SP_NAIH', right_on='N_AIH', how='left').drop('N_AIH', axis=1)

In [25]:
merged_sp_rd.shape

(6267475, 7)

In [93]:
merged_sp_rd = merged_sp_rd[merged_sp_rd['SEXO'].notna()]
merged_sp_rd.shape

(1190649, 10)

In [98]:
df_rd.head()

,N_AIH,SEXO,IDADE,INSTRU
0,3518101448609,3,10,0
1,3518101279825,3,47,0
2,3518101457475,3,25,0
3,3518101457662,3,49,0
4,3518101525246,3,14,0


### Group by AIH Number

In [94]:
group_by_aih = merged_sp_rd.groupby('SP_NAIH').count()['SP_ATOPROF']
group_by_aih

SP_NAIH
3517103370550     7
3517105344490    17
3517105347096    16
3517105347349    24
3517105348724    26
                 ..
3518500587437     1
3518500617148     1
3518500639269     1
3518500649830     1
3518502074208     2
Name: SP_ATOPROF, Length: 85131, dtype: int64

In [95]:
group_by_aih.isna().sum()

0

In [96]:
group_by_aih.describe()

count    85131.000000
mean        13.986080
std          6.843525
min          1.000000
25%         10.000000
50%         14.000000
75%         17.000000
max        400.000000
Name: SP_ATOPROF, dtype: float64

In [97]:
group_by_aih[group_by_aih.where(group_by_aih > 100).notna()] 

SP_NAIH
3517107562881    400
3517126938336    138
3517128148556    177
3517128303216    111
3517128303238    117
3518101249234    115
Name: SP_ATOPROF, dtype: int64

### Group by ATO PROF

In [99]:
group_by_proc = merged_sp_rd.groupby('SP_ATOPROF').count()['SP_NAIH']
group_by_proc

SP_ATOPROF
101020066        6
101020074        1
201010011        9
201010020        1
201010216        1
             ...  
802010032    33988
802010083      393
802010091       98
802010105       28
802010199     3793
Name: SP_NAIH, Length: 633, dtype: int64

In [100]:
group_by_proc.describe()

count      633.000000
mean      1880.962085
std       8411.129658
min          1.000000
25%          3.000000
50%         13.000000
75%        209.000000
max      85083.000000
Name: SP_NAIH, dtype: float64

In [103]:
group_by_proc[group_by_proc.where(group_by_proc > 10000).notna()].sort_values()

SP_ATOPROF
411020013    10154
202050017    10400
202010201    11777
202031110    12217
205020143    13676
211040010    13844
411010026    13851
417010036    13912
417010028    14229
801010047    16510
303100044    16620
211070149    20134
801010039    20772
202120082    20846
301010048    26554
202020541    26692
202020380    29324
802010032    33988
411010034    40071
417010010    41673
202120023    48572
202031179    58202
301010145    60989
310010020    62406
301010170    62649
310010039    66469
211040061    83083
214010040    85083
Name: SP_NAIH, dtype: int64

In [52]:
procedures.query("CODE == '0301010170' ")

,CODE,DESCRIPTION
